In [1]:
import numpy as np
import scipy
%matplotlib notebook
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


# 1.1 Tracking by assignment without particle merging or division

### f) Linear program definition
In order to write down the matrix of constraints, you may find convenient to finish implementing the following function:

In [2]:
### Convert edge list to incidence matrix:
def get_incidence_matrix(edge_list, nb_nodes=None):
    """
    Utility function converting a list of uv-edges into an incidence matrix.
    
    edge_list should be a numpy array of shape (number_of_edges, 2) including the uv-pairs 
    for each edge in the graph
    """
    nb_nodes = edge_list.max() + 1 if nb_nodes is None else nb_nodes
    
    nb_edges = edge_list.shape[0]
    inc_matrix = np.zeros((nb_nodes, nb_edges))
    
    for i, (u,v) in enumerate(edge_list):
        inc_matrix[u][i] = -1
        inc_matrix[v][i] = 1
    
    # delete entries for start and target nodes
    inc_matrix = np.delete(inc_matrix, [0, nb_nodes-1], 0)
    return inc_matrix

In [3]:
def get_min_cost_flow_LP(edge_list, costs):
    A_eq = get_incidence_matrix(edge_list)
    b_eq = np.zeros(edge_list.max() - 1)
    A_leq = np.identity(edge_list.shape[0])
    b_leq = np.ones(edge_list.shape[0])
    bounds = (0, None)

    print(f'Linear Program:\n\n'
          f'min_x c*x\n'
          f's.t. A_eq x=b_eq\n'
          f'A_leq*x<=b_leq\n'
          f'{bounds[0]}<= x <= {bounds[1]}\n\n'
          f'with c = {costs}\n'
          f'A_eq = {A_eq}\n'
          f'b_eq = {b_eq}\n'
          f'A_leq={A_leq}\n'
          f'b_leq = {b_leq}')

    return costs, A_eq, b_eq, A_leq, b_leq, bounds
    
edge_list = np.array([[0, 1], [0, 2], [0, 3], [1, 4], [2, 5], [3, 6], [4, 7], [4 ,8], 
                          [5, 7], [5 ,8], [6, 7], [6 ,8], [7, 9], [8 , 10], [9, 11], [10 ,11], 
                          [11, 12], [12 ,13]])

costs = np.array([0, 0, 0, -10, -20, -18, 5, 26, 16, 5, 41, 20, -13 ,-8, 17, 34, -9, 0])

costs, A_eq, b_eq, A_leq, b_leq, bounds = get_min_cost_flow_LP(edge_list, costs)

### g) Solve the linear program
To solve the LP you may find useful the following wrapper of `scipy.optimize.linprog`.

In [4]:
from scipy.optimize import linprog

def solve_LP(costs, A_eq=None, b_eq=None, A_leq=None, b_leq=None, bounds=(0, None), 
            edge_list=None):
    """
    A wrapper around `scipy.optimize.linprog`.
    
    The `bounds` parameter represents what in the exercise sheet is defined as (x_low, x_high)
    """
    optim_result = scipy.optimize.linprog(costs, A_ub=A_leq, b_ub=b_leq, A_eq=A_eq, 
                                          b_eq=b_eq,  bounds=bounds, method='revised simplex')
    solution = optim_result.x
    assert optim_result.status == 0, "Something went wrong during the optimization"
    
    # Do some printing:
    np.set_printoptions(precision=4)
    print("LP solution: \n", solution)
    print("LP minimum energy: ", optim_result.fun)
    
    # Print selected edges:
    if edge_list is not None:
        assert edge_list.shape[0] == solution.shape[0]
        for i, edge in enumerate(edge_list):
            if np.allclose(solution[i], 1.):
                print("Edge ({},{}) selected".format(edge[0], edge[1]))
    
    return solution, optim_result.fun

In [5]:
def get_N_tot(edge_list, solution):
    target = edge_list.max()
    # set nodes of unused edges to -1
    edge_list[np.repeat(solution[:, np.newaxis], 2, axis=1) == 0] = -1
    N_tot = np.sum(edge_list[:, 1] == target)
    return N_tot

solution, optim_result = solve_LP(costs, A_eq, b_eq, A_leq, b_leq, bounds, np.copy(edge_list))    
N_tot = get_N_tot(np.copy(edge_list), solution)
print(f'N_tot = {N_tot}')

LP solution: 
 [1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1.]
LP minimum energy:  -10.0
Edge (0,1) selected
Edge (1,4) selected
Edge (4,7) selected
Edge (7,9) selected
Edge (9,11) selected
Edge (11,12) selected
Edge (12,13) selected
N_tot = 1


$N_{tot}=1$ as expected, because appearances and disappearances are note allowed except for frame 1 and 3 respectively and frame 3 ends with one observation.  

### h) Particle appearance and disappearance

In [6]:
edge_list = np.append(edge_list, [[0, 7], [0, 8], [0, 11], [4, 13], [5, 13], [6, 13], [9, 13], [10, 13]], axis=0)
costs = np.append(costs[:], [6, 6, 6, 10, 10, 10, 10, 10])

costs, A_eq, b_eq, A_leq, b_leq, bounds = get_min_cost_flow_LP(edge_list, costs)

solution, optim_result = solve_LP(costs, A_eq, b_eq, A_leq, b_leq, bounds, edge_list)    
N_tot = get_N_tot(edge_list[:], solution)
print(f'N_tot = {N_tot}')

LP solution: 
 [1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1.
 1. 1.]
LP minimum energy:  -32.0
Edge (0,1) selected
Edge (0,2) selected
Edge (0,3) selected
Edge (1,4) selected
Edge (2,5) selected
Edge (3,6) selected
Edge (4,7) selected
Edge (5,8) selected
Edge (7,9) selected
Edge (8,10) selected
Edge (11,12) selected
Edge (12,13) selected
Edge (0,11) selected
Edge (6,13) selected
Edge (9,13) selected
Edge (10,13) selected
N_tot = 4


Now the resulting number of particles is $N_{tot}=4$. The maximum distance for a particle to travel between one frame and the next one is ????

# 1.2 Bonus: Tracking with particle merging and/or division

In [ ]:
### Your solution goes here
pass